# Universidad del Valle de Guatemala
## Facultad de Ingeniería
### Departamento de Computación

---

# Laboratorio 2: Series de Tiempo

**Integrantes:**
- Diego Alexander Hernández Silvestre, 21270
- Linda Inés Jiménez Vides, 21169

**Curso:** Data Science  
**Sección:** 10  

---

Guatemala, 1 de agosto de 2024


# DEFINICION DE FUNCIONES

In [47]:
import pandas as pd
import os
import re
import glob

In [10]:
def convertXlsxToCsv(directory):
    print("📂 Iniciando la conversión de archivos .xlsx a .csv...")
    
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.xlsx'):
                xlsxPath = os.path.join(root, file)
                
                try:
                    df = pd.read_excel(xlsxPath, engine='openpyxl')
                    print(f"✅ Cargado: {xlsxPath}")
                except Exception as e:
                    print(f"❌ Error al cargar {xlsxPath}: {e}")
                    continue
                
                csvPath = os.path.splitext(xlsxPath)[0] + '.csv'
                
                try:
                    df.to_csv(csvPath, index=False)
                    print(f"✅ Convertido: {csvPath}")
                except Exception as e:
                    print(f"❌ Error al convertir {xlsxPath} a {csvPath}: {e}")

    print("🏁 Conversión completada.")

directory = './'
convertXlsxToCsv(directory)

📂 Iniciando la conversión de archivos .xlsx a .csv...
✅ Cargado: ./Consumo\CONSUMO-2024-05.xlsx
✅ Convertido: ./Consumo\CONSUMO-2024-05.csv
✅ Cargado: ./Importacion\IMPORTACION-VOLUMEN-2024-05.xlsx
✅ Convertido: ./Importacion\IMPORTACION-VOLUMEN-2024-05.csv
✅ Cargado: ./PreciosPromedioNacionales\Precios-Promedio-Nacionales-Diarios-2024-3.xlsx
✅ Convertido: ./PreciosPromedioNacionales\Precios-Promedio-Nacionales-Diarios-2024-3.csv
🏁 Conversión completada.


In [11]:
def removeEmptyRows(csvPath):
    print("📂 Iniciando la limpieza de filas vacías en el archivo CSV...")
    
    try:
        df = pd.read_csv(csvPath)
        print(f"✅ Archivo cargado: {csvPath}")
    except Exception as e:
        print(f"❌ Error al cargar {csvPath}: {e}")
        return
    
    initialRowCount = len(df)
    df.dropna(how='all', inplace=True)
    finalRowCount = len(df)
    rowsRemoved = initialRowCount - finalRowCount
    
    print(f"🧹 Filas eliminadas: {rowsRemoved}")
    
    try:
        df.to_csv(csvPath, index=False)
        print(f"✅ Archivo actualizado: {csvPath}")
    except Exception as e:
        print(f"❌ Error al guardar {csvPath}: {e}")

    print("🏁 Limpieza completada.")


In [12]:
def removeRows(csvPath, rowsFromStart, rowsFromEnd):
    print("📂 Iniciando la eliminación de filas en el archivo CSV...")

    try:
        df = pd.read_csv(csvPath, skiprows=2, header=None)
        print(f"✅ Archivo cargado y primera fila eliminada: {csvPath}")
    except Exception as e:
        print(f"❌ Error al cargar {csvPath}: {e}")
        return

    df.columns = df.iloc[0]
    df = df[1:]

    initialRowCount = len(df)

    rowsFromStart = min(rowsFromStart, initialRowCount)
    rowsFromEnd = min(rowsFromEnd, initialRowCount - rowsFromStart)

    if rowsFromStart > 0:
        df = df.drop(df.index[:rowsFromStart])
        print(f"🧹 Filas eliminadas del inicio: {rowsFromStart}")

    if rowsFromEnd > 0:
        df = df.drop(df.index[-rowsFromEnd:])
        print(f"🧹 Filas eliminadas del final: {rowsFromEnd}")

    finalRowCount = len(df)
    rowsRemoved = initialRowCount - finalRowCount
    print(f"🧹 Total filas eliminadas: {rowsRemoved}")

    try:
        df.to_csv(csvPath, index=False)
        print(f"✅ Archivo actualizado: {csvPath}")
    except Exception as e:
        print(f"❌ Error al guardar {csvPath}: {e}")

    print("🏁 Eliminación de filas completada.")


In [13]:

def cleanCellLineBreaks(csvPath):
    print("📂 Iniciando la limpieza de saltos de línea dentro de celdas...")

    try:
        # Leer el archivo CSV en un DataFrame
        df = pd.read_csv(csvPath)
        print(f"✅ Archivo cargado: {csvPath}")
    except Exception as e:
        print(f"❌ Error al cargar {csvPath}: {e}")
        return

    # Limpiar los saltos de línea en el encabezado
    df.columns = [re.sub(r'[\n\r]', '', col) if isinstance(col, str) else col for col in df.columns]
    # Limpiar los saltos de línea en las celdas de datos
    df = df.applymap(lambda x: re.sub(r'[\n\r]', '', x) if isinstance(x, str) else x)

    try:
        # Guardar el DataFrame actualizado en el mismo archivo CSV
        df.to_csv(csvPath, index=False)
        print(f"✅ Archivo actualizado después de limpiar saltos de línea: {csvPath}")
    except Exception as e:
        print(f"❌ Error al guardar {csvPath}: {e}")

    print("🏁 Limpieza de saltos de línea completada.")
    df.head()


In [14]:
def convertExcelToCsv(excelPath):
    print("📂 Iniciando la conversión de hojas de Excel a CSV...")

    try:
        # Cargar el archivo Excel
        excel_file = pd.ExcelFile(excelPath)
        print(f"✅ Archivo cargado: {excelPath}")
    except Exception as e:
        print(f"❌ Error al cargar {excelPath}: {e}")
        return
    
    # Iterar sobre cada hoja en el archivo Excel
    for sheet_name in excel_file.sheet_names:
        try:
            # Leer la hoja en un DataFrame
            df = pd.read_excel(excel_file, sheet_name=sheet_name)
            print(f"📄 Hoja cargada: {sheet_name}")
            
            # Definir el nombre del archivo CSV
            csv_file_name = f"PreciosPromedioNacionales/{sheet_name}.csv"
            
            # Guardar el DataFrame como un archivo CSV
            df.to_csv(csv_file_name, index=False)
            print(f"✅ Archivo guardado: {csv_file_name}")
        except Exception as e:
            print(f"❌ Error al procesar la hoja {sheet_name}: {e}")

    print("🏁 Conversión de hojas de Excel a CSV completada.")

In [39]:
def combineHeaders(csvPath):
    print("📂 Iniciando la combinación de encabezados...")

    try:
        # Leer el archivo CSV en un DataFrame sin encabezado
        df = pd.read_csv(csvPath, header=None)
        print(f"✅ Archivo cargado: {csvPath}")
    except Exception as e:
        print(f"❌ Error al cargar {csvPath}: {e}")
        return None
    
    # Comprobar que el DataFrame tiene al menos 2 filas
    if len(df) < 2:
        print("❌ El archivo CSV no tiene suficientes filas para combinar encabezados.")
        return None

    # Crear una nueva lista de encabezados combinados
    new_headers = [
        df.iloc[0, 0],  # FECHA
        f"{df.iloc[0, 1]} {df.iloc[1, 1]}",  # Tipo de Cambio GTQ/USD
        f"{df.iloc[0, 2]} {df.iloc[1, 2]}",  # Superior GTQ/GALON
        f"{df.iloc[0, 3]} {df.iloc[1, 3]}",  # Regular GTQ/GALON
        f"{df.iloc[0, 4]} {df.iloc[1, 4]}",  # Diesel GTQ/GALON
        f"{df.iloc[0, 5]} {df.iloc[1, 5]}",  # Bunker GTQ/GALON
        f"{df.iloc[0, 6]} {df.iloc[1, 6]} {df.iloc[1, 6]}",  # Glp Cilindro 25Lbs. GTQ/CIL 25 LB
        f"{df.iloc[0, 6]} {df.iloc[1, 7]}"  # Glp Cilindro 25Lbs. GTQ/LB
    ]

    # Reasignar los nuevos encabezados al DataFrame
    df.columns = new_headers

    # Eliminar las dos primeras filas del DataFrame
    df = df[2:]

    # Restablecer el índice
    df.reset_index(drop=True, inplace=True)

    try:
        # Guardar el DataFrame actualizado en el mismo archivo CSV
        df.to_csv(csvPath, index=False)
        print(f"✅ Archivo actualizado con encabezados combinados: {csvPath}")
    except Exception as e:
        print(f"❌ Error al guardar {csvPath}: {e}")

    print("🏁 Combinación de encabezados completada.")
    return df

In [48]:
def unifyCSVs(csvFolderPath, outputCsvPath):
    print("📂 Iniciando la unificación de archivos CSV...")

    # Lista para almacenar los DataFrames
    df_list = []

    # Buscar todos los archivos CSV en la carpeta especificada
    csv_files = glob.glob(f"{csvFolderPath}/*.csv")
    
    # Cargar cada archivo CSV en un DataFrame y agregarlo a la lista
    for csv_file in sorted(csv_files):  # Ordenar los archivos
        try:
            df = pd.read_csv(csv_file)
            df_list.append(df)
            print(f"✅ Archivo cargado: {csv_file}")
        except Exception as e:
            print(f"❌ Error al cargar {csv_file}: {e}")

    # Concatenar todos los DataFrames en uno solo
    if df_list:
        combined_df = pd.concat(df_list, ignore_index=True)
        
        # Asegurarse de que la columna 'FECHA' esté en formato datetime
        combined_df['FECHA'] = pd.to_datetime(combined_df['FECHA'], errors='coerce')
        
        # Ordenar el DataFrame por la columna 'FECHA'
        combined_df.sort_values(by='FECHA', inplace=True)

        # Restablecer el índice
        combined_df.reset_index(drop=True, inplace=True)

        # Guardar el DataFrame unificado en el archivo de salida
        try:
            combined_df.to_csv(outputCsvPath, index=False)
            print(f"✅ Archivo unificado guardado: {outputCsvPath}")
        except Exception as e:
            print(f"❌ Error al guardar {outputCsvPath}: {e}")
    else:
        print("❌ No se encontraron archivos CSV para unificar.")

    print("🏁 Unificación de archivos completada.")


# CONSUMO-2024-05.xlsx  

In [15]:
csvPath = 'Consumo/CONSUMO-2024-05.csv'
removeEmptyRows(csvPath)

📂 Iniciando la limpieza de filas vacías en el archivo CSV...
✅ Archivo cargado: Consumo/CONSUMO-2024-05.csv
🧹 Filas eliminadas: 5
✅ Archivo actualizado: Consumo/CONSUMO-2024-05.csv
🏁 Limpieza completada.


In [16]:
csvPath = 'Consumo/CONSUMO-2024-05.csv'
rowsFromStart = 0  
rowsFromEnd = 2    
removeRows(csvPath, rowsFromStart, rowsFromEnd)

📂 Iniciando la eliminación de filas en el archivo CSV...
✅ Archivo cargado y primera fila eliminada: Consumo/CONSUMO-2024-05.csv
🧹 Filas eliminadas del final: 2
🧹 Total filas eliminadas: 2
✅ Archivo actualizado: Consumo/CONSUMO-2024-05.csv
🏁 Eliminación de filas completada.


In [17]:
csvPath = 'Consumo/CONSUMO-2024-05.csv'
cleanCellLineBreaks(csvPath)

df = pd.read_csv(csvPath)
df.head()

📂 Iniciando la limpieza de saltos de línea dentro de celdas...
✅ Archivo cargado: Consumo/CONSUMO-2024-05.csv
✅ Archivo actualizado después de limpiar saltos de línea: Consumo/CONSUMO-2024-05.csv
🏁 Limpieza de saltos de línea completada.


C:\Users\daher\AppData\Local\Temp\ipykernel_8416\851395631.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: re.sub(r'[\n\r]', '', x) if isinstance(x, str) else x)


,Fecha,Aceites lubricantes,Asfalto,Bunker,Ceras,Combustible turbo jet,Diesel bajo azufre,Diesel ultra bajo azufre,Gas licuado de petróleo,Gasolina de aviación,...,Kerosina,Mezclasoleosas,Naftas,PetCoke,Petróleo crudo,Solventes,Butano,Diesel alto azufre,Orimulsión,Total
0,2000-01-01 00:00:00,0.0,48446.0,296766.99,0.0,0.0,0.0,0.0,194410.476190,1426.0,...,64026.09,0.0,0.0,0.0,0.0,0.0,0.0,634667.06,0,1.750545e+06
1,2000-02-01 00:00:00,0.0,50597.0,328116.44,0.0,0.0,0.0,0.0,174710.552381,1458.0,...,62660.30,0.0,0.0,0.0,0.0,0.0,0.0,642380.66,0,1.773220e+06
2,2000-03-01 00:00:00,0.0,27593.0,368589.63,0.0,0.0,0.0,0.0,189234.066667,1503.0,...,61361.99,0.0,0.0,0.0,0.0,0.0,0.0,699807.25,0,1.909499e+06
3,2000-04-01 00:00:00,0.0,53794.0,396300.47,0.0,0.0,0.0,0.0,174330.607143,1561.0,...,61814.32,0.0,0.0,0.0,0.0,0.0,0.0,586803.98,0,1.800933e+06
4,2000-05-01 00:00:00,0.0,60137.0,449368.92,0.0,0.0,0.0,0.0,191745.147619,1642.0,...,54098.32,0.0,0.0,0.0,0.0,0.0,0.0,656948.20,0,1.941772e+06


# IMPORTACION-VOLUMEN-2024-05.xlsx  

In [18]:
csvPath = 'Importacion/IMPORTACION-VOLUMEN-2024-05.csv'
removeEmptyRows(csvPath)

📂 Iniciando la limpieza de filas vacías en el archivo CSV...
✅ Archivo cargado: Importacion/IMPORTACION-VOLUMEN-2024-05.csv
🧹 Filas eliminadas: 5
✅ Archivo actualizado: Importacion/IMPORTACION-VOLUMEN-2024-05.csv
🏁 Limpieza completada.


In [19]:
csvPath = 'Importacion/IMPORTACION-VOLUMEN-2024-05.csv'
rowsFromStart = 0  
rowsFromEnd = 2    
removeRows(csvPath, rowsFromStart, rowsFromEnd)

📂 Iniciando la eliminación de filas en el archivo CSV...
✅ Archivo cargado y primera fila eliminada: Importacion/IMPORTACION-VOLUMEN-2024-05.csv
🧹 Filas eliminadas del final: 2
🧹 Total filas eliminadas: 2
✅ Archivo actualizado: Importacion/IMPORTACION-VOLUMEN-2024-05.csv
🏁 Eliminación de filas completada.


# Precios-Promedio-Nacionales-Diarios-2024-3.xlsx

In [20]:
excelPath = 'PreciosPromedioNacionales/Precios-Promedio-Nacionales-Diarios-2024-3.xlsx'  # Cambia esto por el nombre de tu archivo Excel
convertExcelToCsv(excelPath)

📂 Iniciando la conversión de hojas de Excel a CSV...
✅ Archivo cargado: PreciosPromedioNacionales/Precios-Promedio-Nacionales-Diarios-2024-3.xlsx
📄 Hoja cargada: 2024
✅ Archivo guardado: PreciosPromedioNacionales/2024.csv
📄 Hoja cargada: 2023
✅ Archivo guardado: PreciosPromedioNacionales/2023.csv
📄 Hoja cargada: 2022
✅ Archivo guardado: PreciosPromedioNacionales/2022.csv
📄 Hoja cargada: 2021
✅ Archivo guardado: PreciosPromedioNacionales/2021.csv
🏁 Conversión de hojas de Excel a CSV completada.


In [32]:
removeEmptyRows('PreciosPromedioNacionales/2021.csv')
removeEmptyRows('PreciosPromedioNacionales/2022.csv')
removeEmptyRows('PreciosPromedioNacionales/2023.csv')
removeEmptyRows('PreciosPromedioNacionales/2024.csv')

📂 Iniciando la limpieza de filas vacías en el archivo CSV...
✅ Archivo cargado: PreciosPromedioNacionales/2021.csv
🧹 Filas eliminadas: 0
✅ Archivo actualizado: PreciosPromedioNacionales/2021.csv
🏁 Limpieza completada.
📂 Iniciando la limpieza de filas vacías en el archivo CSV...
✅ Archivo cargado: PreciosPromedioNacionales/2022.csv
🧹 Filas eliminadas: 0
✅ Archivo actualizado: PreciosPromedioNacionales/2022.csv
🏁 Limpieza completada.
📂 Iniciando la limpieza de filas vacías en el archivo CSV...
✅ Archivo cargado: PreciosPromedioNacionales/2023.csv
🧹 Filas eliminadas: 5
✅ Archivo actualizado: PreciosPromedioNacionales/2023.csv
🏁 Limpieza completada.
📂 Iniciando la limpieza de filas vacías en el archivo CSV...
✅ Archivo cargado: PreciosPromedioNacionales/2024.csv
🧹 Filas eliminadas: 12
✅ Archivo actualizado: PreciosPromedioNacionales/2024.csv
🏁 Limpieza completada.


In [34]:
csvPath = 'PreciosPromedioNacionales/2021.csv'
rowsFromStart = 0  
rowsFromEnd = 2    
removeRows(csvPath, rowsFromStart, rowsFromEnd)

📂 Iniciando la eliminación de filas en el archivo CSV...
✅ Archivo cargado y primera fila eliminada: PreciosPromedioNacionales/2021.csv
🧹 Filas eliminadas del final: 2
🧹 Total filas eliminadas: 2
✅ Archivo actualizado: PreciosPromedioNacionales/2021.csv
🏁 Eliminación de filas completada.


In [38]:
csvPath = 'PreciosPromedioNacionales/2022.csv'
rowsFromStart = 0  
rowsFromEnd = 2    
removeRows(csvPath, rowsFromStart, rowsFromEnd)

📂 Iniciando la eliminación de filas en el archivo CSV...
✅ Archivo cargado y primera fila eliminada: PreciosPromedioNacionales/2022.csv
🧹 Filas eliminadas del final: 2
🧹 Total filas eliminadas: 2
✅ Archivo actualizado: PreciosPromedioNacionales/2022.csv
🏁 Eliminación de filas completada.


In [43]:
csvPath = 'PreciosPromedioNacionales/2023.csv'
rowsFromStart = 0  
rowsFromEnd = 2    
removeRows(csvPath, rowsFromStart, rowsFromEnd)

📂 Iniciando la eliminación de filas en el archivo CSV...
✅ Archivo cargado y primera fila eliminada: PreciosPromedioNacionales/2023.csv
🧹 Filas eliminadas del final: 2
🧹 Total filas eliminadas: 2
✅ Archivo actualizado: PreciosPromedioNacionales/2023.csv
🏁 Eliminación de filas completada.


In [45]:
csvPath = 'PreciosPromedioNacionales/2024.csv'
rowsFromStart = 0  
rowsFromEnd = 2    
removeRows(csvPath, rowsFromStart, rowsFromEnd)

📂 Iniciando la eliminación de filas en el archivo CSV...
✅ Archivo cargado y primera fila eliminada: PreciosPromedioNacionales/2024.csv
🧹 Filas eliminadas del final: 2
🧹 Total filas eliminadas: 2
✅ Archivo actualizado: PreciosPromedioNacionales/2024.csv
🏁 Eliminación de filas completada.


In [46]:
combineHeaders('PreciosPromedioNacionales/2021.csv')
combineHeaders('PreciosPromedioNacionales/2022.csv')
combineHeaders('PreciosPromedioNacionales/2023.csv')
combineHeaders('PreciosPromedioNacionales/2024.csv')

📂 Iniciando la combinación de encabezados...
✅ Archivo cargado: PreciosPromedioNacionales/2021.csv
✅ Archivo actualizado con encabezados combinados: PreciosPromedioNacionales/2021.csv
🏁 Combinación de encabezados completada.
📂 Iniciando la combinación de encabezados...
✅ Archivo cargado: PreciosPromedioNacionales/2022.csv
✅ Archivo actualizado con encabezados combinados: PreciosPromedioNacionales/2022.csv
🏁 Combinación de encabezados completada.
📂 Iniciando la combinación de encabezados...
✅ Archivo cargado: PreciosPromedioNacionales/2023.csv
✅ Archivo actualizado con encabezados combinados: PreciosPromedioNacionales/2023.csv
🏁 Combinación de encabezados completada.
📂 Iniciando la combinación de encabezados...
✅ Archivo cargado: PreciosPromedioNacionales/2024.csv
✅ Archivo actualizado con encabezados combinados: PreciosPromedioNacionales/2024.csv
🏁 Combinación de encabezados completada.


,FECHA,Tipo de Cambio GTQ/USD,Superior GTQ/GALON,Regular GTQ/GALON,Diesel GTQ/GALON,Bunker GTQ/GALON,Glp Cilindro 25Lbs. GTQ/CIL 25 LB GTQ/CIL 25 LB,Glp Cilindro 25Lbs. GTQ/LB
0,2024-01-01 00:00:00,7.82702,30.44,28.95,29.47,18.99,115,4.6
1,2024-01-02 00:00:00,7.82481,30.44,28.95,29.47,18.99,115,4.6
2,2024-01-03 00:00:00,7.82445,30.47,28.97,29.48,18.99,115,4.6
3,2024-01-04 00:00:00,7.82409,30.47,28.97,29.48,18.99,115,4.6
4,2024-01-05 00:00:00,7.82322,30.47,28.97,29.48,18.99,115,4.6
...,...,...,...,...,...,...,...,...
205,2024-07-24 00:00:00,NaN,32.79,31.29,28.09,19.19,110,4.4
206,2024-07-25 00:00:00,NaN,32.79,31.29,28.09,19.19,110,4.4
207,2024-07-26 00:00:00,NaN,32.79,31.29,28.09,19.19,110,4.4
208,2024-07-27 00:00:00,NaN,32.79,31.29,28.09,19.19,110,4.4


In [49]:
unifyCSVs('PreciosPromedioNacionales', 'PreciosPromedioNacionales/PreciosPromedioNacionales.csv')

📂 Iniciando la unificación de archivos CSV...
✅ Archivo cargado: PreciosPromedioNacionales\2021.csv
✅ Archivo cargado: PreciosPromedioNacionales\2022.csv
✅ Archivo cargado: PreciosPromedioNacionales\2023.csv
✅ Archivo cargado: PreciosPromedioNacionales\2024.csv
✅ Archivo unificado guardado: PreciosPromedioNacionales/PreciosPromedioNacionales.csv
🏁 Unificación de archivos completada.
